# Preprocesamiento de Datos

## Introducción

El preprocesamiento de datos es una etapa crucial en el desarrollo de cualquier proyecto de aprendizaje automático. Este proceso asegura que los datos estén en un formato adecuado y optimizado para el entrenamiento del modelo. En este notebook, llevaremos a cabo varias transformaciones y mejoras en el conjunto de datos de imágenes de rayos X de rodillas para asegurar la mejor calidad posible para el análisis y entrenamiento de modelos.

Las etapas de preprocesamiento incluyen:

1. Eliminación de imágenes duplicadas
2. Separación de imágenes con dos rodillas
3. Aumento de datos
4. Normalización de valores de píxeles
5. Conversión a escala de grises
6. Redimensionamiento uniforme

Cada una de estas etapas se explicará y justificará en detalle en las secciones siguientes.

### Eliminación de Imágenes Duplicadas

La eliminación de imágenes duplicadas es una etapa importante para asegurar que no haya redundancia en el conjunto de datos. Los duplicados pueden sesgar el entrenamiento del modelo y reducir su capacidad de generalización. Utilizaremos técnicas de hashing para identificar y eliminar imágenes duplicada. 

Después de copiar las imágenes no duplicadas a la carpeta de salida, podemos verificar el nuevo conteo de imágenes por categoría.

In [1]:
import os
import cv2
import hashlib
import pandas as pd

# Definir las rutas correctamente basadas en la ubicación del notebook
base_path = os.path.abspath('')
data_path = os.path.join(base_path, 'data', 'raw', 'images')
output_path = os.path.join(base_path, 'data', 'interim', 'non_duplicates')

# Crear la carpeta de salida si no existe
os.makedirs(output_path, exist_ok=True)

def calculate_image_hash(image):
    """
    Calcula el hash MD5 de una imagen para identificar duplicados.
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return hashlib.md5(image).hexdigest()

def verify_and_copy_non_duplicates(data_path, output_path):
    """
    Verifica la presencia de imágenes duplicadas y copia las no duplicadas a la carpeta de salida.
    """
    image_hashes = {}
    duplicate_images = []
    
    for category in ['normal', 'doubtful', 'mild', 'moderate', 'severe']:
        category_path = os.path.join(data_path, category)
        output_category_path = os.path.join(output_path, category)
        os.makedirs(output_category_path, exist_ok=True)
        
        if not os.path.exists(category_path):
            raise FileNotFoundError(f'La ruta especificada no existe: {category_path}')
        
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            if filename.lower().endswith(('png', 'jpg', 'jpeg')):  # Verificar si el archivo es una imagen
                image = cv2.imread(file_path)
                image_hash = calculate_image_hash(image)
                
                if image_hash in image_hashes:
                    duplicate_images.append((category, filename, image_hashes[image_hash]))
                else:
                    image_hashes[image_hash] = (category, filename)
                    # Copiar imagen no duplicada a la carpeta de salida
                    cv2.imwrite(os.path.join(output_category_path, filename), image)
    
    duplicate_images_df = pd.DataFrame(duplicate_images, columns=['category', 'filename', 'original'])
    
    return duplicate_images_df

# Ejecutar la función
duplicate_images_df = verify_and_copy_non_duplicates(data_path, output_path)

### Verificación del Nuevo Conteo de Imágenes por Categoría

def count_images_per_category(path):
    """
    Cuenta la cantidad de imágenes por categoría en la ruta especificada.
    """
    category_counts = {'normal': 0, 'doubtful': 0, 'mild': 0, 'moderate': 0, 'severe': 0}
    for category in category_counts.keys():
        category_path = os.path.join(path, category)
        if os.path.exists(category_path):
            category_counts[category] = len(os.listdir(category_path))
    return category_counts

# Contar imágenes por categoría antes de eliminar duplicados
pre_duplicate_counts = count_images_per_category(data_path)
print("Conteo de imágenes por categoría antes de eliminar duplicados:")
display(pd.DataFrame.from_dict(pre_duplicate_counts, orient='index', columns=['count']))

# Contar imágenes por categoría después de eliminar duplicados
post_duplicate_counts = count_images_per_category(output_path)
print("Conteo de imágenes por categoría después de eliminar duplicados:")
display(pd.DataFrame.from_dict(post_duplicate_counts, orient='index', columns=['count']))

# Contar duplicados por categoría
duplicate_counts = duplicate_images_df['category'].value_counts().sort_index()

# Mostrar número de duplicados y cuántas se eliminaron por categoría
print("Número de imágenes duplicadas por categoría:")
display(duplicate_counts)

Conteo de imágenes por categoría antes de eliminar duplicados:


,count
normal,514
doubtful,477
mild,232
moderate,221
severe,206


Conteo de imágenes por categoría después de eliminar duplicados:


,count
normal,482
doubtful,469
mild,219
moderate,203
severe,206


Número de imágenes duplicadas por categoría:


category
doubtful     8
mild        13
moderate    18
normal      32
Name: count, dtype: int64

### Identificación y Separación de Imágenes con Dos Rodillas

En esta sección, se separan las imágenes que contienen dos rodillas en dos imágenes individuales, una por cada rodilla. Este paso es crucial para estandarizar el dataset y asegurar que cada imagen contenga una única rodilla, facilitando así el procesamiento posterior y la clasificación.

In [2]:
import os
import cv2
import pandas as pd

# Definir las rutas correctamente basadas en la ubicación del notebook
base_path = os.path.abspath('')
input_path = os.path.join(base_path, 'data', 'interim', 'non_duplicates')
output_path = os.path.join(base_path, 'data', 'interim', 'separated_knees')

# Crear la carpeta de salida si no existe
os.makedirs(output_path, exist_ok=True)

def get_prefix_and_suffix(category):
    """
    Retorna el prefijo y el sufijo basado en la categoría.
    """
    category_dict = {
        'normal': 'NormalG0',
        'doubtful': 'DoubtfulG1',
        'mild': 'MildG2',
        'moderate': 'ModerateG3',
        'severe': 'SevereG4'
    }
    return category_dict.get(category, '')

def count_and_separate_knees(input_path, output_path):
    """
    Identifica y separa las imágenes con dos rodillas.
    """
    knee_count_data = {'normal': 0, 'doubtful': 0, 'mild': 0, 'moderate': 0, 'severe': 0}
    two_knees_data = {'normal': 0, 'doubtful': 0, 'mild': 0, 'moderate': 0, 'severe': 0}
    
    for category in knee_count_data.keys():
        category_path = os.path.join(input_path, category)
        output_category_path = os.path.join(output_path, category)
        os.makedirs(output_category_path, exist_ok=True)
        
        prefix = get_prefix_and_suffix(category)
        count = 1
        
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            image = cv2.imread(file_path)
            height, width = image.shape[:2]
            
            if width == 640:  # Imágenes con dos rodillas
                two_knees_data[category] += 1
                
                # Separar las dos rodillas
                left_knee = image[:, :320]
                right_knee = image[:, 320:]
                
                left_knee_filename = f"{prefix} ({count})_left.png"
                right_knee_filename = f"{prefix} ({count + 1})_right.png"
                
                cv2.imwrite(os.path.join(output_category_path, left_knee_filename), left_knee)
                cv2.imwrite(os.path.join(output_category_path, right_knee_filename), right_knee)
                count += 2
            else:
                knee_count_data[category] += 1
                new_filename = f"{prefix} ({count}).png"
                cv2.imwrite(os.path.join(output_category_path, new_filename), image)
                count += 1
    
    return knee_count_data, two_knees_data

# Ejecutar la función y mostrar los resultados
knee_count_data, two_knees_data = count_and_separate_knees(input_path, output_path)

print("Conteo de imágenes con una rodilla por categoría:")
display(pd.DataFrame.from_dict(knee_count_data, orient='index', columns=['count']))

print("Conteo de imágenes con dos rodillas por categoría:")
display(pd.DataFrame.from_dict(two_knees_data, orient='index', columns=['count']))

Conteo de imágenes con una rodilla por categoría:


,count
normal,426
doubtful,434
mild,182
moderate,165
severe,206


Conteo de imágenes con dos rodillas por categoría:


,count
normal,56
doubtful,35
mild,37
moderate,38
severe,0


### Aumento de Datos (Data Augmentation)

El aumento de datos (Data Augmentation) se utilizará para generar más imágenes en las categorías menos representadas.

Al hacer aumento de datos (data augmentation) en imágenes de radiografías de rodillas, es crucial preservar las características médicas importantes y evitar distorsiones que puedan afectar la interpretación médica. Aquí hay algunos criterios importantes a tener en cuenta:

1. No Rotar las Imágenes: Mantener la orientación original de las rodillas.
2. No Cambiar la Proporción de Aspecto: Preservar las proporciones originales para no distorsionar las características anatómicas.
3. Ajuste de Brillo y Contraste: Permitir variaciones que podrían ocurrir debido a diferentes condiciones de escaneo.
4. Añadir Ruido Gaussian: Imitar el ruido que podría estar presente en diferentes máquinas de escaneo.
5. Ajuste de Escala Límite: Permitir un ligero cambio en la escala sin alterar las proporciones anatómicas.
6. Desplazamiento Horizontal y Vertical: Permitir un ligero desplazamiento que podría ocurrir en diferentes condiciones de escaneo.
7. Cortar y Pegar Pequeñas Porciones de la Imagen: Imitar posibles variaciones sin alterar la estructura general.

In [3]:
import os
import cv2
import hashlib
import numpy as np
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from IPython.display import display
import torch

# Definir las rutas correctamente basadas en la ubicación del notebook
base_path = os.path.abspath('')
input_path = os.path.join(base_path, 'data', 'interim', 'separated_knees')
output_path = os.path.join(base_path, 'data', 'interim', 'augmented')

# Crear la carpeta de salida si no existe
os.makedirs(output_path, exist_ok=True)

# Definir el aumento de datos con Albumentations
augmentation = A.Compose([
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),
        A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.2)
    ], p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.HorizontalFlip(p=0.5),
    ToTensorV2()
])

def get_prefix_and_suffix(category):
    """
    Retorna el prefijo y el sufijo basado en la categoría.
    """
    category_dict = {
        'normal': 'NormalG0',
        'doubtful': 'DoubtfulG1',
        'mild': 'MildG2',
        'moderate': 'ModerateG3',
        'severe': 'SevereG4'
    }
    return category_dict.get(category, '')

def calculate_image_hash(image):
    """
    Calcula el hash MD5 de una imagen para identificar duplicados.
    """
    return hashlib.md5(image).hexdigest()

def augment_data(input_path, output_path, target_count=500):
    """
    Realiza el aumento de datos para balancear las clases.
    """
    category_counts = {category: len(os.listdir(os.path.join(input_path, category))) for category in os.listdir(input_path)}
    image_hashes = set()
    
    for category in category_counts.keys():
        category_path = os.path.join(input_path, category)
        output_category_path = os.path.join(output_path, category)
        os.makedirs(output_category_path, exist_ok=True)
        
        images = []
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            if filename.lower().endswith(('png', 'jpg', 'jpeg')):  # Verificar si el archivo es una imagen
                image = cv2.imread(file_path)
                image_hash = calculate_image_hash(image)
                image_hashes.add(image_hash)
                images.append((filename, image))
                output_filepath = os.path.join(output_category_path, filename)
                cv2.imwrite(output_filepath, image)  # Guardar la imagen original también
        
        prefix = get_prefix_and_suffix(category)
        existing_count = category_counts[category]
        
        # Generar imágenes aumentadas
        if len(images) > 0:
            while category_counts[category] < target_count:
                for i, (filename, image) in enumerate(images):
                    augmented = augmentation(image=image)["image"]
                    aug_image_hash = calculate_image_hash(augmented.numpy().transpose(1, 2, 0))
                    if aug_image_hash not in image_hashes:
                        aug_filename = f"{prefix} ({existing_count + 1})_augmented.png"
                        aug_filepath = os.path.join(output_category_path, aug_filename)
                        cv2.imwrite(aug_filepath, augmented.numpy().transpose(1, 2, 0))  # Convertir tensor a imagen
                        category_counts[category] += 1
                        existing_count += 1
                        image_hashes.add(aug_image_hash)
                    if category_counts[category] >= target_count:
                        break
    
    return category_counts

# Ejecutar la función y mostrar los resultados
initial_counts = {category: len(os.listdir(os.path.join(input_path, category))) for category in os.listdir(input_path)}
augmented_counts = augment_data(input_path, output_path)

# Mostrar el conteo inicial de imágenes por categoría
print("Conteo inicial de imágenes por categoría:")
display(pd.DataFrame.from_dict(initial_counts, orient='index', columns=['count']))

# Mostrar el conteo de imágenes después del aumento de datos por categoría
print("Conteo después del aumento de datos por categoría:")
display(pd.DataFrame.from_dict(augmented_counts, orient='index', columns=['count']))

Conteo inicial de imágenes por categoría:


,count
doubtful,504
mild,256
moderate,241
normal,538
severe,206


Conteo después del aumento de datos por categoría:


,count
doubtful,504
mild,500
moderate,500
normal,538
severe,500


### Normalización de Imágenes

La normalización es un paso crucial en el preprocesamiento de datos que consiste en escalar los valores de los píxeles de las imágenes a un rango estándar, como 0-1 o -1 a 1. Esto ayuda a mejorar la convergencia y el rendimiento de los algoritmos de machine learning y deep learning.

In [4]:
import os
import cv2
import numpy as np

# Definir las rutas correctamente basadas en la ubicación del notebook
base_path = os.path.abspath('')
input_path = os.path.join(base_path, 'data', 'interim', 'augmented')
output_path = os.path.join(base_path, 'data', 'interim', 'normalized')

# Crear la carpeta de salida si no existe
os.makedirs(output_path, exist_ok=True)

def normalize_image(image):
    """
    Normaliza la imagen a valores entre 0 y 1.
    """
    return image / 255.0

def normalize_images(input_path, output_path):
    """
    Normaliza las imágenes y las guarda en la carpeta de salida.
    """
    for category in os.listdir(input_path):
        category_path = os.path.join(input_path, category)
        output_category_path = os.path.join(output_path, category)
        os.makedirs(output_category_path, exist_ok=True)
        
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            normalized_image = normalize_image(image)
            output_file_path = os.path.join(output_category_path, filename)
            cv2.imwrite(output_file_path, (normalized_image * 255).astype(np.uint8))

# Ejecutar la función de normalización
normalize_images(input_path, output_path)
print("Imágenes normalizadas y guardadas en la carpeta 'normalized'.")

Imágenes normalizadas y guardadas en la carpeta 'normalized'.


#### Revisión de Dimensionalidad de las Imágenes

Verificar el número de canales que presenta cada una de las imágenes. En la etapa de verificación de la calidad de los datos, se identificó que las imágenes estaban a color.

In [5]:
import os
import cv2
import pandas as pd

# Definir las rutas correctamente basadas en la ubicación del notebook
base_path = os.path.abspath('')
input_path = os.path.join(base_path, 'data', 'interim', 'normalized')

def check_image_channels(input_path):
    """
    Verifica el número de canales en cada imagen y clasifica si es blanco y negro o color.
    """
    image_data = []
    
    for category in os.listdir(input_path):
        category_path = os.path.join(input_path, category)
        if os.path.isdir(category_path):
            for filename in os.listdir(category_path):
                if filename.lower().endswith(('png', 'jpg', 'jpeg')):  # Verificar si el archivo es una imagen
                    file_path = os.path.join(category_path, filename)
                    image = cv2.imread(file_path)
                    if image is not None:
                        channels = image.shape[2] if len(image.shape) == 3 else 1
                        color_type = 'Color' if channels == 3 else 'Blanco y Negro'
                        image_data.append((category, filename, color_type))
    
    return pd.DataFrame(image_data, columns=['Category', 'Filename', 'Color Type'])

# Ejecutar la función
image_color_df = check_image_channels(input_path)

# Mostrar el resultado
print("Resumen de imágenes por tipo de color:")
print(image_color_df['Color Type'].value_counts())

print("\nDetalle de imágenes:")
display(image_color_df)

Resumen de imágenes por tipo de color:
Color Type
Color    2542
Name: count, dtype: int64

Detalle de imágenes:


,Category,Filename,Color Type
0,doubtful,DoubtfulG1 (1).png,Color
1,doubtful,DoubtfulG1 (10).png,Color
2,doubtful,DoubtfulG1 (100).png,Color
3,doubtful,DoubtfulG1 (101).png,Color
4,doubtful,DoubtfulG1 (102).png,Color
...,...,...,...
2537,severe,SevereG4 (95).png,Color
2538,severe,SevereG4 (96).png,Color
2539,severe,SevereG4 (97).png,Color
2540,severe,SevereG4 (98).png,Color


## Conversión a Escala de Grises

La conversión a escala de grises es un paso esencial en el preprocesamiento de imágenes médicas, ya que reduce la complejidad de los datos sin perder información relevante para el análisis. Convertir las imágenes a escala de grises para simplificar el procesamiento y mejorar la eficiencia computacional.

Las imágenes en escala de grises tienen menos dimensiones que las imágenes en color (1 canal en lugar de 3), lo que reduce el tiempo de procesamiento y la memoria requerida, sin comprometer la calidad de la información para las tareas de diagnóstico y análisis.

In [6]:
# Definir las rutas correctamente basadas en la ubicación del notebook
input_path = os.path.join(base_path, 'data', 'interim', 'normalized')
output_path = os.path.join(base_path, 'data', 'interim', 'grayscale')

# Crear la carpeta de salida si no existe
os.makedirs(output_path, exist_ok=True)

def convert_to_grayscale(image):
    """
    Convierte una imagen a escala de grises.
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def grayscale_images(input_path, output_path):
    """
    Convierte las imágenes a escala de grises y las guarda en la carpeta de salida.
    """
    for category in os.listdir(input_path):
        category_path = os.path.join(input_path, category)
        output_category_path = os.path.join(output_path, category)
        os.makedirs(output_category_path, exist_ok=True)
        
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            image = cv2.imread(file_path)
            grayscale_image = convert_to_grayscale(image)
            output_file_path = os.path.join(output_category_path, filename)
            cv2.imwrite(output_file_path, grayscale_image)

# Ejecutar la función de conversión a escala de grises
grayscale_images(input_path, output_path)
print("Imágenes convertidas a escala de grises y guardadas en la carpeta 'grayscale'.")

Imágenes convertidas a escala de grises y guardadas en la carpeta 'grayscale'.


## Redimensionamiento Uniforme

El redimensionamiento uniforme asegura que todas las imágenes tengan el mismo tamaño, lo cual es crucial para el entrenamiento de modelos de aprendizaje automático. Redimensionar todas las imágenes a un tamaño uniforme para garantizar la compatibilidad y eficiencia en el procesamiento posterior.

Los modelos de aprendizaje automático requieren que las imágenes de entrada tengan dimensiones uniformes para poder procesarlas en lotes (batch processing). Además, un tamaño uniforme mejora la consistencia y el rendimiento del modelo.

In [7]:
import os
import cv2
import pandas as pd
from IPython.display import display, HTML
import base64

# Definir las rutas correctamente basadas en la ubicación del notebook
base_path = os.path.abspath('')
input_path = os.path.join(base_path, 'data', 'interim', 'grayscale')
interim_output_path = os.path.join(base_path, 'data', 'interim', 'resized')
final_output_path = os.path.join(base_path, 'data', 'processed', 'images')

# Crear las carpetas de salida si no existen
os.makedirs(interim_output_path, exist_ok=True)
os.makedirs(final_output_path, exist_ok=True)

def resize_image(image, size=(256, 256)):
    """
    Redimensiona una imagen al tamaño especificado.
    """
    return cv2.resize(image, size)

def resize_images(input_path, interim_output_path):
    """
    Redimensiona las imágenes y las guarda en las carpetas de salida.
    """
    for category in os.listdir(input_path):
        category_path = os.path.join(input_path, category)
        interim_output_category_path = os.path.join(interim_output_path, category)
        os.makedirs(interim_output_category_path, exist_ok=True)
        
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            if filename.lower().endswith(('png', 'jpg', 'jpeg')):  # Verificar si el archivo es una imagen
                image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                resized_image = resize_image(image)
                interim_output_file_path = os.path.join(interim_output_category_path, filename)
                cv2.imwrite(interim_output_file_path, resized_image)

# Ejecutar la función de redimensionamiento
resize_images(input_path, interim_output_path)
print("Imágenes redimensionadas y guardadas en la carpeta 'interim/resized'.")

Imágenes redimensionadas y guardadas en la carpeta 'interim/resized'.


### Revisión Final y Almacenamiento

En esta sección, se realizará una revisión final de las imágenes preprocesadas y se almacenarán en la carpeta final para su uso en el entrenamiento de modelos. Asegurar que todas las imágenes preprocesadas estén en la forma y calidad deseada y almacenarlas en la carpeta final de datos procesados.

Una revisión final asegura que todas las imágenes están correctamente preprocesadas y listas para su uso en los modelos de machine learning, garantizando la consistencia y calidad del conjunto de datos.

In [8]:
# Definir las rutas correctamente basadas en la ubicación del notebook
base_path = os.path.abspath('')
input_path = os.path.join(base_path, 'data', 'interim', 'resized')
final_output_path = os.path.join(base_path, 'data', 'processed', 'images')

# Crear la carpeta de salida si no existe
os.makedirs(final_output_path, exist_ok=True)

def final_review_and_store_images(input_path, final_output_path):
    """
    Realiza una revisión final de las imágenes y las almacena en la carpeta final.
    """
    for category in os.listdir(input_path):
        category_path = os.path.join(input_path, category)
        final_output_category_path = os.path.join(final_output_path, category)
        os.makedirs(final_output_category_path, exist_ok=True)
        
        for filename in os.listdir(category_path):
            file_path = os.path.join(category_path, filename)
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            
            # Realizar cualquier revisión final necesaria (por ejemplo, verificación de dimensiones)
            if image.shape == (256, 256):
                output_file_path = os.path.join(final_output_category_path, filename)
                cv2.imwrite(output_file_path, image)

# Ejecutar la función de revisión final y almacenamiento
final_review_and_store_images(input_path, final_output_path)
print("Imágenes revisadas y almacenadas en la carpeta 'processed/images'.")

def count_images_per_category(path):
    """
    Cuenta la cantidad de imágenes por categoría en la ruta especificada.
    """
    category_counts = {'normal': 0, 'doubtful': 0, 'mild': 0, 'moderate': 0, 'severe': 0}
    for category in category_counts.keys():
        category_path = os.path.join(path, category)
        if os.path.exists(category_path):
            category_counts[category] = len(os.listdir(category_path))
    return category_counts

# Contar imágenes por categoría después de la revisión final
final_counts = count_images_per_category(final_output_path)
print("Conteo final de imágenes por categoría:")
display(pd.DataFrame.from_dict(final_counts, orient='index', columns=['count']))

Imágenes revisadas y almacenadas en la carpeta 'processed/images'.
Conteo final de imágenes por categoría:


,count
normal,538
doubtful,504
mild,500
moderate,500
severe,500


### Revisión Adicional de Duplicados

Verificación final de duplicados en cada una de las etapas del preprocesamiento

In [9]:
import os
import cv2
import hashlib
import pandas as pd

def calculate_image_hash(image):
    """
    Calcula el hash MD5 de una imagen para identificar duplicados.
    """
    return hashlib.md5(image).hexdigest()

def verify_duplicates(data_path):
    """
    Verifica la presencia de imágenes duplicadas en la carpeta especificada.
    """
    image_hashes = {}
    duplicate_images = []
    
    for category in os.listdir(data_path):
        category_path = os.path.join(data_path, category)
        if os.path.isdir(category_path):  # Verificar si es un directorio
            for filename in os.listdir(category_path):
                file_path = os.path.join(category_path, filename)
                if filename.lower().endswith(('png', 'jpg', 'jpeg')):  # Verificar si el archivo es una imagen
                    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                    image_hash = calculate_image_hash(image)
                    
                    if image_hash in image_hashes:
                        duplicate_images.append((category, filename, image_hashes[image_hash]))
                    else:
                        image_hashes[image_hash] = (category, filename)
    
    duplicate_images_df = pd.DataFrame(duplicate_images, columns=['category', 'filename', 'original'])
    return duplicate_images_df

# Verificación de duplicados para cada etapa
data_paths = [
    os.path.join(base_path, 'data', 'raw', 'images'),
    os.path.join(base_path, 'data', 'interim', 'non_duplicates'),
    os.path.join(base_path, 'data', 'interim', 'separated_knees'),
    os.path.join(base_path, 'data', 'interim', 'augmented'),
    os.path.join(base_path, 'data', 'interim', 'normalized'),
    os.path.join(base_path, 'data', 'interim', 'grayscale'),
    os.path.join(base_path, 'data', 'interim', 'resized'),
    os.path.join(base_path, 'data', 'processed', 'images')
]

for path in data_paths:
    print(f"Verificando duplicados en {path}:")
    duplicate_images_df = verify_duplicates(path)
    if not duplicate_images_df.empty:
        print(f"Se encontraron duplicados en {path}")
        display(duplicate_images_df)
    else:
        print(f"No se encontraron duplicados en {path}")

Verificando duplicados en C:\Users\gluna\Desktop\SIC\SIC_2023_Capstone\Github_Repo\Knee_Arthritis_Detection\data\raw\images:
Se encontraron duplicados en C:\Users\gluna\Desktop\SIC\SIC_2023_Capstone\Github_Repo\Knee_Arthritis_Detection\data\raw\images


,category,filename,original
0,doubtful,DoubtfulG1 (332).png,"(doubtful, DoubtfulG1 (329).png)"
1,doubtful,DoubtfulG1 (386).png,"(doubtful, DoubtfulG1 (383).png)"
2,doubtful,DoubtfulG1 (453).png,"(doubtful, DoubtfulG1 (447).png)"
3,doubtful,DoubtfulG1 (466).png,"(doubtful, DoubtfulG1 (450).png)"
4,doubtful,DoubtfulG1 (76).png,"(doubtful, DoubtfulG1 (73).png)"
...,...,...,...
66,normal,NormalG0 (474).png,"(normal, NormalG0 (466).png)"
67,normal,NormalG0 (475).png,"(normal, NormalG0 (457).png)"
68,normal,NormalG0 (476).png,"(normal, NormalG0 (462).png)"
69,normal,NormalG0 (477).png,"(normal, NormalG0 (464).png)"


Verificando duplicados en C:\Users\gluna\Desktop\SIC\SIC_2023_Capstone\Github_Repo\Knee_Arthritis_Detection\data\interim\non_duplicates:
No se encontraron duplicados en C:\Users\gluna\Desktop\SIC\SIC_2023_Capstone\Github_Repo\Knee_Arthritis_Detection\data\interim\non_duplicates
Verificando duplicados en C:\Users\gluna\Desktop\SIC\SIC_2023_Capstone\Github_Repo\Knee_Arthritis_Detection\data\interim\separated_knees:
No se encontraron duplicados en C:\Users\gluna\Desktop\SIC\SIC_2023_Capstone\Github_Repo\Knee_Arthritis_Detection\data\interim\separated_knees
Verificando duplicados en C:\Users\gluna\Desktop\SIC\SIC_2023_Capstone\Github_Repo\Knee_Arthritis_Detection\data\interim\augmented:
No se encontraron duplicados en C:\Users\gluna\Desktop\SIC\SIC_2023_Capstone\Github_Repo\Knee_Arthritis_Detection\data\interim\augmented
Verificando duplicados en C:\Users\gluna\Desktop\SIC\SIC_2023_Capstone\Github_Repo\Knee_Arthritis_Detection\data\interim\normalized:
No se encontraron duplicados en C:\Use